### Import Libraries

In [2]:
# import libraries 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pulp as plp
from pulp import *
from sklearn import preprocessing
import seaborn as sns

### Read the Dataset

In [3]:
#read csv file
df = pd.read_csv('all_prd.csv')
df

,Player,Team,Rating,Role,prd_pnts
0,AR Patel,Delhi Capitals,8.5,All Rounder,89.066
1,BA Stokes,Rajasthan Royals,8.5,All Rounder,2.772
2,S Gopal,Rajasthan Royals,8.5,All Rounder,25.582
3,R Ashwin,Delhi Capitals,9.0,All Rounder,39.764
4,AM Rahane,Delhi Capitals,8.0,Batsman,-2.330
5,JC Buttler,Rajasthan Royals,9.5,Wicketkeeper batsman,9.008
6,MP Stoinis,Delhi Capitals,8.5,Batsman,40.826
7,PP Shaw,Delhi Capitals,8.0,Batsman,21.268
8,R Parag,Rajasthan Royals,8.0,Batsman,26.244
9,RV Uthappa,Rajasthan Royals,9.0,Batsman,14.906


In [8]:
#assign team columns dynamically
team_num=df['Team']
#get unique value of teams
team_num=team_num.unique()

### Assign each feature to players
- create Dummies for Columns :- Team and Role
- create a dictonary with index as player names and value as features

In [9]:
#a function to create dictory of each player with their features
def transform_data(players_df):
    
    # copy
    df = players_df.copy()

    #create dummies for the columns Team and Role  
    df = pd.get_dummies(df, columns=['Team', 'Role'])
    
    
    #assign the feature columns according to team ,role , predicted points and ratings
    feature_cols = ['Rating', "Team_"+team_num[0], "Team_"+team_num[1], 
               'Role_All Rounder', 'Role_Batsman', 'Role_Bowler', 'Role_Wicketkeeper batsman','prd_pnts']


    # check if all features are present
    for col in feature_cols:
        if col not in df.columns:
            raise("Required columns missing to form features:", col)
            
    # Creates a list of the Players
    player_names = list(df['Player'])
    feat_dict = {}
    for col in feature_cols:
        feat_dict[col] = dict(zip(player_names, df[col].values))
    return player_names, feat_dict
    


### Call the function on our data

In [10]:
player_names, feat = transform_data(df)
feat#printing the dictonary 

{'Rating': {'AR Patel': 8.5,
  'BA Stokes': 8.5,
  'S Gopal': 8.5,
  'R Ashwin': 9.0,
  'AM Rahane': 8.0,
  'JC Buttler': 9.5,
  'MP Stoinis': 8.5,
  'PP Shaw': 8.0,
  'R Parag': 8.0,
  'RV Uthappa': 9.0,
  'S Dhawan': 10.0,
  'SPD Smith': 8.0,
  'SS Iyer': 10.0,
  'SV Samson': 9.0,
  'A Nortje': 9.0,
  'JD Unadkat': 8.5,
  'JC Archer': 10.0,
  'R Tewatia': 9.0,
  'K Rabada': 9.5},
 'Team_Delhi Capitals': {'AR Patel': 1,
  'BA Stokes': 0,
  'S Gopal': 0,
  'R Ashwin': 1,
  'AM Rahane': 1,
  'JC Buttler': 0,
  'MP Stoinis': 1,
  'PP Shaw': 1,
  'R Parag': 0,
  'RV Uthappa': 0,
  'S Dhawan': 1,
  'SPD Smith': 0,
  'SS Iyer': 1,
  'SV Samson': 0,
  'A Nortje': 1,
  'JD Unadkat': 0,
  'JC Archer': 0,
  'R Tewatia': 0,
  'K Rabada': 1},
 'Team_Rajasthan Royals': {'AR Patel': 0,
  'BA Stokes': 1,
  'S Gopal': 1,
  'R Ashwin': 0,
  'AM Rahane': 0,
  'JC Buttler': 1,
  'MP Stoinis': 0,
  'PP Shaw': 0,
  'R Parag': 1,
  'RV Uthappa': 1,
  'S Dhawan': 0,
  'SPD Smith': 1,
  'SS Iyer': 0,
  'SV S

In [11]:
def optimize_team(player_names, features):
    
    
    # Players chosen 
    player_chosen = LpVariable.dicts("playerChosen", player_names, 0, 1, cat='Integer')
     
    # define np problem
    prob = LpProblem("Fantasy_Cricket", LpMaximize)

    # The objective function is added to 'prob' first
    prob += lpSum([feat['prd_pnts'][i]*player_chosen[i] for i in player_names])
    
    # max credits: credits are multiplied by 2 to convert them into integers
    prob += lpSum([feat['Rating'][i]*player_chosen[i] for i in player_names]) <= 100, "MaxCredits"
    
  
    # Wk
    prob += lpSum([feat['Role_Wicketkeeper batsman'][f] * player_chosen[f] for f in player_names]) == 1, "Wkequal"

    # Batsmen count has to be minimum 3 and maximum 5 
    prob += lpSum([feat['Role_Batsman'][f] * player_chosen[f] for f in player_names]) >= 3, "BatsmenMinimum"
    prob += lpSum([feat['Role_Batsman'][f] * player_chosen[f] for f in player_names]) <= 5, "BatsmenMaximum"

    # Bowler count has to be minimum 3 and maximum 5 
    prob += lpSum([feat['Role_Bowler'][f] * player_chosen[f] for f in player_names]) >= 3, "BowlerMinimum"
    prob += lpSum([feat['Role_Bowler'][f] * player_chosen[f] for f in player_names]) <= 5, "BowlerMaximum"

    # All rounder count has to be minimum 1 and maximum 3
    prob += lpSum([feat['Role_All Rounder'][f] * player_chosen[f] for f in player_names]) >= 1, "ArMinimum"
    prob += lpSum([feat['Role_All Rounder'][f] * player_chosen[f] for f in player_names]) <= 3, "ArMaximum"

    # Team 1 minimum and maximum players
    prob += lpSum([feat["Team_"+team_num[0]][f] * player_chosen[f] for f in player_names]) >= 4, "Team1Minimum"
    prob += lpSum([feat["Team_"+team_num[0]][f] * player_chosen[f] for f in player_names]) <= 7, "Team1Maximum"

    # Team 2 minimum and maximum player
    prob += lpSum([feat["Team_"+team_num[1]][f] * player_chosen[f] for f in player_names]) >= 4, "Team2Minimum"
    prob += lpSum([feat["Team_"+team_num[1]][f] * player_chosen[f] for f in player_names]) <= 7, "Team2Maximum"

    # The problem data is written to an .lp file
    r = prob.writeLP("FantasyCricket.lp")
    
    prob.solve()
                       
    print("Status:", LpStatus[prob.status])
    # prob.solver
    #maximize the objective function
    print("maximized = {}".format(round(value(prob.objective))))
    
    
    return prob

### Call the optimization function 

In [12]:

prob = optimize_team(player_names, feat)
prob#print the optimization function , constraints and variables 

Status: Optimal
maximized = 440


Fantasy_Cricket:
MAXIMIZE
-2.33*playerChosen_AM_Rahane + 89.066*playerChosen_AR_Patel + 36.75*playerChosen_A_Nortje + 2.772*playerChosen_BA_Stokes + 49.856*playerChosen_JC_Archer + 9.008*playerChosen_JC_Buttler + 0.092*playerChosen_JD_Unadkat + 67.806*playerChosen_K_Rabada + 40.826*playerChosen_MP_Stoinis + 21.268*playerChosen_PP_Shaw + 14.905999999999999*playerChosen_RV_Uthappa + 39.764*playerChosen_R_Ashwin + 26.244*playerChosen_R_Parag + -3.5639999999999996*playerChosen_R_Tewatia + 3.448*playerChosen_SPD_Smith + 33.44*playerChosen_SS_Iyer + 8.948*playerChosen_SV_Samson + 19.62*playerChosen_S_Dhawan + 25.581999999999997*playerChosen_S_Gopal + 0.0
SUBJECT TO
MaxCredits: 8 playerChosen_AM_Rahane + 8.5 playerChosen_AR_Patel
 + 9 playerChosen_A_Nortje + 8.5 playerChosen_BA_Stokes
 + 10 playerChosen_JC_Archer + 9.5 playerChosen_JC_Buttler
 + 8.5 playerChosen_JD_Unadkat + 9.5 playerChosen_K_Rabada
 + 8.5 playerChosen_MP_Stoinis + 8 playerChosen_PP_Shaw
 + 9 playerChosen_RV_Uthappa + 9 play

### Display the chosen player 
- We display the chosen player to satisfy all the constraints and meet the objective 
- choosing the best 11 players 

In [13]:
player_roles = dict(zip(df['Player'], df['Role']))#make a dictonary of the obtained player's name and role

players_choosen = []#list to store the player names
players_credits = []#list to store the rating
players_prd_points=[]#list to store the fantasy points
players_roles_lst=[]
for v in prob.variables():#loop for all the variables
    if (v.value() >0):#check if they are selected
        act_name = " ".join(v.name.split('_')[-2:])
        credit = feat['Rating'][act_name]
        pnt=feat['prd_pnts'][act_name]
        players_prd_points.append(pnt)#append them
        players_choosen.append(act_name)
        players_roles_lst.append(player_roles[act_name])
        players_credits.append(credit)
        print(v.name, ' = ', credit, player_roles[act_name],pnt)#print them 
        
print('Total credits:', sum(players_credits))#print the total credit points

playerChosen_AR_Patel  =  8.5 All Rounder 89.066
playerChosen_A_Nortje  =  9.0 Bowler 36.75
playerChosen_JC_Archer  =  10.0 Bowler 49.856
playerChosen_JC_Buttler  =  9.5 Wicketkeeper batsman 9.008
playerChosen_K_Rabada  =  9.5 Bowler 67.806
playerChosen_MP_Stoinis  =  8.5 Batsman 40.826
playerChosen_PP_Shaw  =  8.0 Batsman 21.268
playerChosen_R_Ashwin  =  9.0 All Rounder 39.764
playerChosen_R_Parag  =  8.0 Batsman 26.244
playerChosen_SS_Iyer  =  10.0 Batsman 33.44
playerChosen_S_Gopal  =  8.5 All Rounder 25.581999999999997
Total credits: 98.5


In [14]:
new_data=pd.DataFrame()
new_data["Players"]=players_choosen
new_data["Rating"]=players_credits
new_data["Fantasy points"]=players_prd_points
new_data["Roles"]=players_roles_lst

In [15]:
new_data

,Players,Rating,Fantasy points,Roles
0,AR Patel,8.5,89.066,All Rounder
1,A Nortje,9.0,36.750,Bowler
2,JC Archer,10.0,49.856,Bowler
3,JC Buttler,9.5,9.008,Wicketkeeper batsman
4,K Rabada,9.5,67.806,Bowler
5,MP Stoinis,8.5,40.826,Batsman
6,PP Shaw,8.0,21.268,Batsman
7,R Ashwin,9.0,39.764,All Rounder
8,R Parag,8.0,26.244,Batsman
9,SS Iyer,10.0,33.440,Batsman


In [16]:
new_data.to_csv("Predicted_team.csv")